In [62]:
import pandas as pd
import os

## Load data

In [63]:
load_data = pd.read_csv("./Data/Load_hourly_2050.csv")
load_data['date'] = load_data.apply(lambda x:pd.to_datetime(x['*Hour']-1, origin='2050-01-01',unit='h'),axis=1)
load_data

,*Hour,Load,date
0,1,26927.58882,2050-01-01 00:00:00
1,2,26008.34278,2050-01-01 01:00:00
2,3,25292.68962,2050-01-01 02:00:00
3,4,24895.77208,2050-01-01 03:00:00
4,5,24933.29478,2050-01-01 04:00:00
...,...,...,...
8755,8756,31840.06796,2050-12-31 19:00:00
8756,8757,30699.46724,2050-12-31 20:00:00
8757,8758,29651.53631,2050-12-31 21:00:00
8758,8759,28576.43091,2050-12-31 22:00:00


### define seasons

In [64]:
seasons = ['Winter', 'Spring', 'Summer', 'Fall']
def define_season(date):
    if date < pd.Timestamp('2050-03-01'): return 'Winter'
    elif ((date >= pd.Timestamp('2050-03-01')) and (date < pd.Timestamp('2050-06-01'))):
        return 'Spring'
    elif ((date >= pd.Timestamp('2050-06-01')) and (date < pd.Timestamp('2050-09-01'))):
        return 'Summer'
    elif ((date >= pd.Timestamp('2050-09-01')) and (date < pd.Timestamp('2050-12-01'))):
        return 'Fall'
    else: return 'Winter'
load_data['Season'] = load_data['date'].apply(define_season)
load_data[load_data.Season == 'Spring']

,*Hour,Load,date,Season
1416,1417,25404.15077,2050-03-01 00:00:00,Spring
1417,1418,24560.06866,2050-03-01 01:00:00,Spring
1418,1419,24135.54237,2050-03-01 02:00:00,Spring
1419,1420,23865.03363,2050-03-01 03:00:00,Spring
1420,1421,24152.63950,2050-03-01 04:00:00,Spring
...,...,...,...,...
3619,3620,33572.39351,2050-05-31 19:00:00,Spring
3620,3621,34039.41975,2050-05-31 20:00:00,Spring
3621,3622,33187.42436,2050-05-31 21:00:00,Spring
3622,3623,31106.16149,2050-05-31 22:00:00,Spring


## Nuclear, Hydro and other renewables

In [65]:
nuclear_gen = pd.read_csv('./Data/Nucl_hourly_2019.csv')
hydro_gen = pd.read_csv('./Data/lahy_hourly_2019.csv')
otre_gen = pd.read_csv('./Data/otre_hourly_2019.csv')
load_data = load_data.merge(right=otre_gen, on='*Hour')
load_data = load_data.merge(right=hydro_gen, on='*Hour')
load_data = load_data.merge(right=nuclear_gen, on='*Hour')
load_data

,*Hour,Load,date,Season,OtherRenewables,LargeHydro,Nuclear
0,1,26927.58882,2050-01-01 00:00:00,Winter,1789.273657,2350.859949,2272.823279
1,2,26008.34278,2050-01-01 01:00:00,Winter,1778.907508,2359.651647,2273.720723
2,3,25292.68962,2050-01-01 02:00:00,Winter,1775.303300,2063.741773,2274.351674
3,4,24895.77208,2050-01-01 03:00:00,Winter,1772.696067,1929.003648,2274.184531
4,5,24933.29478,2050-01-01 04:00:00,Winter,1776.056231,1978.703105,2273.921185
...,...,...,...,...,...,...,...
8755,8756,31840.06796,2050-12-31 19:00:00,Winter,1485.612960,3244.618784,2271.531560
8756,8757,30699.46724,2050-12-31 20:00:00,Winter,1481.543916,3170.790833,2272.050907
8757,8758,29651.53631,2050-12-31 21:00:00,Winter,1470.839286,2873.841091,2272.360738
8758,8759,28576.43091,2050-12-31 22:00:00,Winter,1405.591229,2262.388558,2274.959260


## Define worst case scenario

In [66]:
scenario_dir = './resilience_scenarios/'
os.makedirs(scenario_dir, exist_ok=True)
durations = [6,12,24,72]

for duration in durations:
    ld = load_data.copy()
    duration_dir = os.path.join(scenario_dir,f'seasonpeak_outage_{duration}h/')
    os.makedirs(duration_dir, exist_ok=True)
    for season in seasons: 
        peak_hour_id = load_data[load_data.Season == season]['Load'].idxmax()
        peak_hour = load_data.loc[peak_hour_id,'date']
        print(f"Peak hour for season {season} is {peak_hour}")
    
        
        time_delta = pd.Timedelta(duration/2, unit='h')
        oneh = pd.Timedelta(1, unit='h')
        ld.loc[ (ld.date>=peak_hour-time_delta+oneh) & (ld.date<=peak_hour+time_delta), 'OtherRenewables'] = 0
        ld.loc[ (ld.date>=peak_hour-time_delta+oneh) & (ld.date<=peak_hour+time_delta), 'LargeHydro'] = 0
        ld.loc[ (ld.date>=peak_hour-time_delta+oneh) & (ld.date<=peak_hour+time_delta), 'Nuclear'] = 0
    ld.to_csv(os.path.join(duration_dir,f'load_gen_{duration}h.csv'))

    


    

Peak hour for season Winter is 2050-12-16 18:00:00
Peak hour for season Spring is 2050-04-24 19:00:00
Peak hour for season Summer is 2050-08-15 17:00:00
Peak hour for season Fall is 2050-09-03 17:00:00
Peak hour for season Winter is 2050-12-16 18:00:00
Peak hour for season Spring is 2050-04-24 19:00:00
Peak hour for season Summer is 2050-08-15 17:00:00
Peak hour for season Fall is 2050-09-03 17:00:00
Peak hour for season Winter is 2050-12-16 18:00:00
Peak hour for season Spring is 2050-04-24 19:00:00
Peak hour for season Summer is 2050-08-15 17:00:00
Peak hour for season Fall is 2050-09-03 17:00:00
Peak hour for season Winter is 2050-12-16 18:00:00
Peak hour for season Spring is 2050-04-24 19:00:00
Peak hour for season Summer is 2050-08-15 17:00:00
Peak hour for season Fall is 2050-09-03 17:00:00
